# 업무자동화를 위한 파이썬 입문 CAMP #10-01-02


> 주제 : 네이버에서 특정 키워드로 상위 노출된 블로그 리스트 출력하기

## Prerequisite


```shell
$ pip install requests
$ pip install beautifulsoup4
```

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
# 만약, "신사동" 이라는 키워드로 네이버에서 검색하면, 어떤 주소로 이동될까요?
# => https://search.naver.com/search.naver?where=nexearch&query=신사동&sm=top_hty&fbm=1&ie=utf8
#
# 만약, "신사동 원룸" 이라는 키워드로 네이버에서 검색하면, 어떤 주소로 이동될까요?
# => https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&ie=utf8&query=신사동+원룸
#
# 즉 어떤 키워드로 검색하던 간에 네이버에서는 `query=[키워드A+키워드B+키워드C]` 라는 주소가 뒤에 붙음을 알 수 있습니다.
# ( 개발자들이 사용하는 표현으로 바꾸면 다음과 같습니다: 네이버 검색에서는 query 라는 parameter의 value로 "키워드A+키워드B+키워드C" 를 받습니다. )


# 먼저, 네이버에서 "신사동 원룸" 이라는 키워드로 한번 검색을 해보겠습니다.
response = requests.get("https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&ie=utf8&query=신사동+원룸")
assert response.status_code is 200

In [3]:
dom = BeautifulSoup(response.content, "html.parser")

blog_post_elements = dom.select('li.sh_blog_top')

for blog_post_element in blog_post_elements:
    title = blog_post_element.select('a.sh_blog_title')[0].attrs.get('title')
    blog_name = blog_post_element.select('a.txt84')[0].text
    url = blog_post_element.select('a.sh_blog_title')[0].attrs.get('href')
    print((title, blog_name, url))

('신사동 원룸 월세 압구정역 하우스테디베어', '(주)피플부동산&최연호', 'http://blog.naver.com/uppdb?Redirect=Log&logNo=220598517114')
('[강남구]용달이사 강남 신사동 역삼동 원룸이사', '★ 이 사 전 문 ☎ 1 6 7 0 - 9 9 8 6 ★', 'http://blog.daum.net/yk_6060/592513')
('역삼동V신사동용달이사X청담동원룸이사(보관이사)', '젊은일꾼들 세진24스토리', 'http://blog.daum.net/transporter2424/1060')
('(강남구)용달이사 (강남)원룸이사 논현동 신사동', '★ 이 사 전 문 ☎ 1 6 7 0 - 9 9 8 6 ★', 'http://blog.daum.net/yk_6060/592492')
('신사동 강남구청역 인근 원룸을 소개하겠습니다', 'ㄴ영달이네~^^*', 'http://blog.naver.com/white_uns2?Redirect=Log&logNo=220578775492')


In [4]:
# 여러 키워드에 대해서 한꺼번에 블로그 포스트 리스트를 불러와보겠습니다.

keywords = [
    "신사동 원룸",
    "신사역 원룸",
    "가로수길 원룸",
    "가로수길 오피스텔",
    "신사역 부동산",
]


for keyword in keywords:
    # 우리가 작성한 키워드는 " " ( 띄어쓰기 ) 로 되어 있지만,
    # 네이버에서 검색을 할 떄는 " " => "+" 로 변경되서 검색되어야 합니다.
    
    query = keyword.replace(' ', '+')
    
    response = requests.get("https://search.naver.com/search.naver?query={query}".format(query=query))
    assert response.status_code is 200


    dom = BeautifulSoup(response.content, "html.parser")

    blog_post_elements = dom.select('li.sh_blog_top')

    for blog_post_element in blog_post_elements:
        title = blog_post_element.select('a.sh_blog_title')[0].attrs.get('title')
        blog_name = blog_post_element.select('a.txt84')[0].text
        url = blog_post_element.select('a.sh_blog_title')[0].attrs.get('href')
        print((keyword, title, blog_name, url))


('신사동 원룸', '신사동 원룸 월세 압구정역 하우스테디베어', '(주)피플부동산&최연호', 'http://blog.naver.com/uppdb?Redirect=Log&logNo=220598517114')
('신사동 원룸', '[강남구]용달이사 강남 신사동 역삼동 원룸이사', '★ 이 사 전 문 ☎ 1 6 7 0 - 9 9 8 6 ★', 'http://blog.daum.net/yk_6060/592513')
('신사동 원룸', '역삼동V신사동용달이사X청담동원룸이사(보관이사)', '젊은일꾼들 세진24스토리', 'http://blog.daum.net/transporter2424/1060')
('신사동 원룸', '(강남구)용달이사 (강남)원룸이사 논현동 신사동', '★ 이 사 전 문 ☎ 1 6 7 0 - 9 9 8 6 ★', 'http://blog.daum.net/yk_6060/592492')
('신사동 원룸', '신사동 강남구청역 인근 원룸을 소개하겠습니다', 'ㄴ영달이네~^^*', 'http://blog.naver.com/white_uns2?Redirect=Log&logNo=220578775492')
('신사역 원룸', '신사역 오피스텔 원룸 샤르망s 월세', '아름다운 세상 TEL.02-555-4472 (24시간)', 'http://blog.daum.net/arangsin/670')
('신사역 원룸', '신사역 오피스텔 고층뷰 원룸 크레신타워 월세', 'House Channel 02.566.2027', 'http://blog.daum.net/gangnamjeil/1317')
('신사역 원룸', '신사역 원룸 학동공원 인근 넓고 보안좋은곳', '하우스마켓', 'http://blog.daum.net/gnhmk/1720')
('가로수길 원룸', '논현동 원룸 가로수길 빌라 T3하우스', '(주)피플부동산&최연호', 'http://blog.naver.com/uppdb?Redirect=Log&logNo=220608134940')
('가로수길 원룸', '강남원룸전세 신사

> 수고하셨습니다. 감사합니다. - 안수찬 올림.